### Inicio

In [1]:
# IMPORTS
import re
import sys
import pandas as pd
from pathlib import Path
import unicodedata

In [2]:
# PATHS
PROJECT_ROOT = Path(r"C:\Users\dsgal\Documents\Finanças\personal_finance_app")
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("✅ Project ready")

✅ Project ready


In [3]:
# LOAD COM SEU CÓDIGO
from src.data.abn.load_abn import load_all_abn
df_raw = load_all_abn()
print(f"✅ Loaded {len(df_raw)} raw transactions")

BANK_TOKENS = ['IBAN', 'BIC', 'ABNA', 'ABNANL', 'INGB', 'RABO', 'COBA', 'CITI', 'NL']

def clean_basic_description(desc: str) -> str:
    """Hard cleaning: uppercase + remove special chars/numbers + bank tokens + PRESERVA ACENTOS."""
    if pd.isna(desc):
        return ""
    
    desc = str(desc).upper()
    
    # Remove bank tokens
    for token in BANK_TOKENS:
        desc = desc.replace(token, ' ')
    
    # NORMALIZA ACENTOS (ã→A, ç→C, é→E)
    desc = unicodedata.normalize('NFKD', desc).encode('ASCII', 'ignore').decode('ASCII')
    
    # Remove só caracteres especiais de verdade
    desc = re.sub(r'[^A-Z\s]', ' ', desc)
    desc = re.sub(r'\s+', ' ', desc).strip()
    return desc

✅ Loaded 798 raw transactions


### Inicio do df_work

In [4]:
# CRIAR df_work a partir de df_raw (EXATO o que você quer)
df_work = df_raw[['description']].copy().reset_index()
df_work = df_work.rename(columns={'index': 'original_index'})
df_work['description_cleaned'] = ""  # ← VAZIA para preencher depois

In [5]:
# Preenche com hard clean (opcional)
df_work['description_cleaned'] = df_work['description'].apply(clean_basic_description)

s_desc = df_work['description_cleaned']

### Apple

In [6]:
def clean_apple_pay(desc_clean: str) -> str:
    if pd.isna(desc_clean): return ""
    desc = str(desc_clean)
    if "BEA APPLE PAY" in desc:
        desc = desc.split("BEA APPLE PAY", 1)[1].strip()
    if "PAS NR" in desc:
        desc = desc.split("PAS NR", 1)[0].strip()
    return re.sub(r'\s+', ' ', desc).strip()

In [7]:
df_apple = df_work[s_desc.str.contains('BEA APPLE PAY', na=False)].copy()
df_apple['description_cleaned'] = df_apple['description_cleaned'].apply(clean_apple_pay)
df_apple.head()

,original_index,description,description_cleaned
1,1,"BEA, Apple Pay A MANTEIGARIA...",A MANTEIGARIA
3,3,"BEA, Apple Pay MINIPRECO,PAS...",MINIPRECO
7,7,"BEA, Apple Pay MINIPRECO,PAS...",MINIPRECO
8,8,"BEA, Apple Pay MINIPRECO,PAS...",MINIPRECO
11,11,"BEA, Apple Pay MINIPRECO,PAS...",MINIPRECO


### Account Balance

In [8]:
df_balance = df_work[s_desc.str.contains('ACCOUNT BALANCE', na=False)].copy()
df_balance['description_cleaned'] = "CREDIT INTEREST"
df_balance.head()

,original_index,description,description_cleaned
161,161,ACCOUNT BALANCED CREDIT INTERE...,CREDIT INTEREST
164,164,ACCOUNT BALANCED CREDIT INTERE...,CREDIT INTEREST
171,171,ACCOUNT BALANCED CREDIT INTERE...,CREDIT INTEREST
174,174,ACCOUNT BALANCED CREDIT INTERE...,CREDIT INTEREST
179,179,ACCOUNT BALANCED CREDIT INTERE...,CREDIT INTEREST


### Basic Package

In [9]:
df_basicPackage = df_work[s_desc.str.contains('BASIC PACKAGE', na=False)].copy()
df_basicPackage['description_cleaned'] = "BASIC PACKAGE"
df_basicPackage.head()

,original_index,description,description_cleaned
40,40,ABN AMRO Bank N.V. Basic Package...,BASIC PACKAGE
129,129,ABN AMRO Bank N.V. Basic Package...,BASIC PACKAGE
223,223,ABN AMRO Bank N.V. Basic Package...,BASIC PACKAGE
324,324,ABN AMRO Bank N.V. Basic Package...,BASIC PACKAGE
388,388,ABN AMRO Bank N.V. Basic Package...,BASIC PACKAGE


### Revolut

In [10]:
df_revolut = df_work[s_desc.str.contains('REVOLUT', na=False)].copy()
df_revolut['description_cleaned'] = "REVOLUT"
df_revolut.head()

,original_index,description,description_cleaned
0,0,"eCom, Apple Pay Revolut**4460...",REVOLUT
16,16,"eCom, Apple Pay Revolut**4460...",REVOLUT
23,23,"eCom, Apple Pay Revolut**4460...",REVOLUT
25,25,"eCom, Apple Pay Revolut**4460...",REVOLUT
41,41,"eCom, Apple Pay Revolut**4460...",REVOLUT


### Tikkie

In [11]:
def clean_tikkie(desc_cleaned: str) -> str:
    if pd.isna(desc_cleaned): return ""
    desc = str(desc_cleaned)
   
    # PADRÃO 1: SEPA OVERBOEKING (pula OMSCHRIJVING até TIKKIE ID)
    if "SEPA OVERBOEKING" in desc and "TIKKIE ID" in desc:
        # Pula até TIKKIE ID (ignora OMSCHRIJVING)
        part = desc.split("TIKKIE ID", 1)[1].strip()
        commas = part.split(",")
        if len(commas) >= 3:
            busca1 = commas[0].strip()  # "COSTELADA DO JUNIOR TESTE"
            busca2 = commas[1].strip()  # "VAN JAIR DE SOUZA JUNIOR"
            if busca2.startswith("VAN"):
                busca2 = busca2[3:].strip()
            return f"{busca1} ({busca2})"
    
    # PADRÃO 2: SEPA IDEAL (igual)
    if "SEPA IDEAL" in desc:
        busca1 = ""
        if "OMSCHRIJVING" in desc and "KENMERK" in desc:
            busca1 = desc.split("OMSCHRIJVING", 1)[1].split("KENMERK", 1)[0].strip()
        busca2 = ""
        if "NAAM" in desc and "VIA TIKKIE" in desc:
            busca2 = desc.split("NAAM", 1)[1].split("VIA TIKKIE", 1)[0].strip()
        if busca1 and busca2:
            return f"{busca1} ({busca2})"
   
    return desc

TIKKIE_TOKENS = [
    # Estrutura SEPA
    "SEPA", "OVERBOEKING", "INCASSO", "IDEAL", "BETAALVERZOEK",
    "A NAAM", "NAAM", "IBAN", "BIC", "ABNANL2A", "KENMERK", "NOTPROVIDED",
    "ECOM", "APPLE", "PAY", "BEA", "PAS NR","A NAME", "ID", "BETAALPAS", "CSID", "NAME",
    
    # Tikkie específico  
    "TIKKIE", "OMSCHRIJVING", "TIKKIE ID", "VIA TIKKIE", "TERUGBOEKING", "TRANSACTIE",
    
    # Banco/Instituição
    "AAB", "INZ", "ABNA", "ABN", "RABO", "INGB", "ALGEMEEN", "MARF E DEUT DEUT", "LU BCIRLULL", 
    "DOORLOPEND", "ZZZ", "E DEUT", "CFROM TO DIRECT SAVINGS FOR INTEREST RATES PLEASE VISIT WWW MRO RENTE",
    "AMRO BANK NV", "FACTUUR", "BE TRWIBEB", "OMSCHRIJVING", "PAKKET", "PAKKETPOLISNR", "BNGH",
    
    # Metadados
    "KENMERK", "MACHTIGING", "EREF", "REMI", "ADYB", "BOFA BOFA","AXXX", "MARF A","DE NTSBDEB XXX", "LU PPLXLUL XXX",
    "TRTP", "INCASSANT", "PERIODE", "NR H IB PVV", "COAXX", "ORDP RKJX E DR R", "TERMIJN TERMIJN (VERVALDATUM MEI BETALINGSREGELING DEUT)",

    # Lugares
    "DUBLIN", "LAND", "NL", "ST", "CCV", "TVM", "CSO", "BCK", "NX"
]

def clean_tikkie_v2(desc_cleaned: str) -> str:
    if pd.isna(desc_cleaned):
        return ""
    desc = str(desc_cleaned)

    # Remove tokens
    tmp = " " + desc + " "
    for token in TIKKIE_TOKENS:
        tmp = tmp.replace(" " + token + " ", " ")
    tmp = re.sub(r"\s+", " ", tmp).strip()
    
    # 🆕 REMOVE LETRAS SOLTAS (exceto dentro de parênteses)
    # Primeiro: protege conteúdo dentro de ()
    tmp = re.sub(r'\(([^)]*)\)', r'@@@PROTECTED@@@ \1 @@@PROTECTED@@@', tmp)
    
    # Remove letras soltas A B C D (1 letra isolada)
    tmp = re.sub(r'\b[A-Z]\b(?!\s*\()', ' ', tmp)
    
    # Restaura conteúdo protegido
    tmp = re.sub(r'@@@PROTECTED@@@', '', tmp)
    
    # Normaliza espaços
    tmp = re.sub(r'\s+', ' ', tmp).strip()
    
    # LÓGICA VAN/VIA
    words = tmp.split()
    for i, word in enumerate(words):
        if word.upper() == "VAN":
            label = " ".join(words[:i]).strip()
            name = " ".join(words[i+1:]).strip()
            if label and name:
                return f"{label} ({name})"
        elif word.upper() == "VIA":
            label = " ".join(words[i+1:]).strip()
            name = " ".join(words[:i]).strip()
            if label and name:
                return f"{label} ({name})"
    
    return tmp




In [12]:
df_teste = df_work.copy()
df_teste['description_cleaned'] = df_teste['description_cleaned'].apply(clean_tikkie_v2)
df_teste.head()

,original_index,description,description_cleaned
0,0,"eCom, Apple Pay Revolut**4460...",REVOLUT IRL
1,1,"BEA, Apple Pay A MANTEIGARIA...",MANTEIGARIA LISBOA PRT
2,2,SEPA Overboeking IBAN: NL28ABN...,SOARES GALDINO CJ
3,3,"BEA, Apple Pay MINIPRECO,PAS...",MINIPRECO LISB PRT
4,4,SEPA Incasso algemeen doorlopend Incassant: NL...,NS GROEP IZ NS REIZIGERS VOOR SOARES GALDINO CJ


In [13]:
df_tikkie = df_work[s_desc.str.contains('TIKKIE', na=False)].copy()
df_tikkie['description_cleaned'] = df_tikkie['description_cleaned'].apply(clean_tikkie)
df_tikkie.head()

,original_index,description,description_cleaned
30,30,SEPA Overboeking IBAN: NL13ABN...,SEPA OVERBOEKING A NAAM AAB INZ TIKKIE OMSCHRI...
34,34,SEPA iDEAL IBAN: NL13ABN...,MASSAGEM DO ATLETA (IKEDA FALEIROS)
37,37,SEPA iDEAL IBAN: NL13ABN...,CHURRASCO (BASTOS BRUSCH)
44,44,SEPA iDEAL IBAN: NL13ABN...,FRANGO (DOS SANTOS)
45,45,SEPA iDEAL IBAN: NL13ABN...,NINA (PAI O)


### SEPA

In [14]:
def clean_sepa(desc_clean: str) -> str:
    if pd.isna(desc_clean): return ""
    desc = str(desc_clean)
    if "NAAM" in desc:
        desc = desc.split("NAAM", 1)[1].strip()
    if "MACHTIGING" in desc:
        desc = desc.split("MACHTIGING", 1)[0].strip()
    if "OMSCHRIJVING" in desc:
        desc = desc.split("OMSCHRIJVING", 1)[0].strip()
    if "KENMERK" in desc:
        desc = desc.split("KENMERK", 1)[1].strip()
    return re.sub(r'\s+', ' ', desc).strip()

In [15]:
df_sepa = df_work[s_desc.str.contains('SEPA', na=False) & ~s_desc.str.contains('TIKKIE', na=False)].copy()
df_sepa['description_cleaned'] = df_sepa['description_cleaned'].apply(clean_sepa)
df_sepa.head()

,original_index,description,description_cleaned
2,2,SEPA Overboeking IBAN: NL28ABN...,D SOARES GALDINO CJ
4,4,SEPA Incasso algemeen doorlopend Incassant: NL...,NS GROEP IZ NS REIZIGERS
5,5,SEPA Incasso algemeen doorlopend Incassant: NL...,ABN AMRO BANK NV
6,6,SEPA Incasso algemeen doorlopend Incassant: NL...,CZ GROEP ZORGVERZEKERAAR
9,9,SEPA Incasso algemeen doorlopend Incassant: NL...,INSHARED NEDERLAND B V


### BEA BETAALPAS

In [16]:
def clean_bea(desc_clean: str) -> str:
    if pd.isna(desc_clean): return ""
    desc = str(desc_clean)
    if "BEA BETAALPAS" in desc:
        desc = desc.split("BEA BETAALPAS", 1)[1].strip()
    if "PAS NR" in desc:
        desc = desc.split("PAS NR", 1)[0].strip()
    return re.sub(r'\s+', ' ', desc).strip()

In [17]:
df_bea = df_work[s_desc.str.contains('BEA BETAALPAS', na=False)].copy()
df_bea['description_cleaned'] = df_bea['description_cleaned'].apply(clean_bea)
df_bea.head()

,original_index,description,description_cleaned
42,42,"BEA, Betaalpas CCV*R.O.J. Po...",CCV R O J POL T H O D
49,49,"BEA, Betaalpas BORENT ENSCHE...",BORENT ENSCHEDE
68,68,"BEA, Betaalpas Action 1370\S...",ACTION SCHUTTERSV
95,95,"BEA, Betaalpas DB Automaten,...",DB AUTOMATEN
370,370,"BEA, Betaalpas Tango Wilp A1...",TANGO WILP A


### Juntar todos os dfs no df_work

In [18]:
# for df_group in [df_tikkie, df_apple, df_basicPackage, df_balance, df_sepa, df_revolut, df_bea]:
#         if len(df_group) > 0:
#             df_work.loc[df_group['original_index'], 'description_cleaned'] = df_group['description_cleaned']

for df_group in [df_teste]:
         if len(df_group) > 0:
             df_work.loc[df_group['original_index'], 'description_cleaned'] = df_group['description_cleaned']



In [19]:
df_work.head()

,original_index,description,description_cleaned
0,0,"eCom, Apple Pay Revolut**4460...",REVOLUT IRL
1,1,"BEA, Apple Pay A MANTEIGARIA...",MANTEIGARIA LISBOA PRT
2,2,SEPA Overboeking IBAN: NL28ABN...,SOARES GALDINO CJ
3,3,"BEA, Apple Pay MINIPRECO,PAS...",MINIPRECO LISB PRT
4,4,SEPA Incasso algemeen doorlopend Incassant: NL...,NS GROEP IZ NS REIZIGERS VOOR SOARES GALDINO CJ


In [20]:
# SALVAR RESULTADO FINAL (SEU CÓDIGO)
TESTS_DIR = Path(r"C:\Users\dsgal\Documents\Finanças\personal_finance_app\tests")
TESTS_DIR.mkdir(exist_ok=True)
    
# Salva df_work completo (com original_index)
df_work.to_csv(TESTS_DIR / 'descriptions_cleaned.csv', index=False)
print(f"✅ SALVO: {TESTS_DIR / 'descriptions_cleaned.csv'}")

✅ SALVO: C:\Users\dsgal\Documents\Finanças\personal_finance_app\tests\descriptions_cleaned.csv
